---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing
from sklearn.cluster import KMeans

def blight_model():
    addresses = pd.read_csv('addresses.csv', engine='python') 
    latlons = pd.read_csv('latlons.csv', engine='python')
    latlon_add = pd.merge(addresses,latlons,how='inner',left_on='address',right_on='address')

    df = pd.read_csv('train.csv', engine='python')
    test = pd.read_csv('test.csv', engine='python')

    df.dropna(subset=['compliance'], inplace=True)

    kmeans = KMeans(n_clusters=8)
    latlon_clean = latlon_add.dropna(subset=['lat'])
    kmeans.fit(latlon_clean[['lat','lon']])
    latlon_clean['cluster_label'] = kmeans.fit_predict(latlon_clean[['lat','lon']])
    
    test_agencies = list(set(test['agency_name']))
    df = df[df['agency_name'].isin(test_agencies)]

    df_with_add = pd.merge(df, latlon_clean, how='inner', left_on='ticket_id', right_on='ticket_id')
    df_with_add['hearing'] = df_with_add['hearing_date'].apply(lambda x: 1 if type(x)==str else 0)
    df_with_add['zip_code'] = df_with_add['zip_code'].fillna(value='0')
    df_with_add['zip_code'] = df_with_add['zip_code'].apply(lambda x: int(x) if x.isdigit() and len(x)==5 else 0)    
    y = df_with_add['compliance']
    x_cols = ['agency_name', 'fine_amount', 'judgment_amount', 'discount_amount', 'cluster_label','hearing','zip_code']
    X = df_with_add[x_cols]
    le = preprocessing.LabelEncoder()
    agency_one_hot = X['agency_name'].to_frame().apply(le.fit_transform)
    enc = preprocessing.OneHotEncoder()
    enc.fit(agency_one_hot)
    onehotlabels = enc.transform(agency_one_hot).toarray()
    
    agency_names = pd.DataFrame(onehotlabels)
    X.reset_index(inplace=True, drop=True)
    sorted_agencies = sorted(list(set(X['agency_name'])))
    agency_cols =['agency_{}'.format(i) for i in range(len(sorted_agencies))]
    agency_map = dict(zip(sorted_agencies,agency_cols))
    agency_names.columns = list(agency_map.values())
    
    revised_x = pd.concat([X, agency_names], axis=1).drop(['agency_name'], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(revised_x.as_matrix(), y.as_matrix())
    clf = GradientBoostingClassifier().fit(X_train, y_train)
    
    test_with_add = pd.merge(test, latlon_clean, how='left', left_on='ticket_id', right_on='ticket_id')
    test_with_add['hearing'] = test_with_add['hearing_date'].apply(lambda x: 1 if type(x)==str else 0)
    test_with_add['zip_code'] = test_with_add['zip_code'].fillna(value='0') 
    test_with_add['zip_code'] = test_with_add['zip_code'].apply(lambda x: int(x) if x.isdigit() and len(x)==5 else 0)
    test_cols = ['agency_name', 'fine_amount','ticket_id','judgment_amount', 'discount_amount', 'cluster_label', 'hearing', 'zip_code']

    test = test_with_add[test_cols]
    test.fillna(test.mean(),inplace=True)
    zeros = [0]*len(agency_cols)
    c = dict(zip(agency_cols, zeros))
    agency_dummies = pd.DataFrame(0, index=np.arange(len(test)), columns=agency_cols)
    merged = pd.concat([test,agency_dummies], axis=1)
    for k,v in agency_map.items():
        merged.loc[merged['agency_name'] == k, v] = 1.0
    merged.drop(['agency_name'], axis=1, inplace=True)
    merged.set_index('ticket_id', inplace=True, drop=True)
    del merged.index.name
    
    probs = pd.DataFrame(clf.predict_proba(merged))
    ans = probs[1]
    ans.index = merged.index
    return ans

In [9]:
blight_model()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


284932    0.087961
285362    0.031177
285361    0.098547
285338    0.133412
285346    0.153834
285345    0.145421
285347    0.129927
285342    0.117493
285530    0.034095
284989    0.042300
285344    0.119019
285343    0.051797
285340    0.048741
285341    0.112466
285349    0.081111
285348    0.074834
284991    0.052186
285532    0.054064
285406    0.046751
285001    0.039065
285006    0.034090
285405    0.033444
285337    0.033870
285496    0.094110
285497    0.101844
285378    0.032726
285589    0.039781
285585    0.087424
285501    0.104652
285581    0.033514
            ...   
376367    0.033615
376366    0.054763
376362    0.058906
376363    0.088610
376365    0.033457
376364    0.054512
376228    0.071134
376265    0.062597
376286    0.046561
376320    0.047262
376314    0.047972
376327    0.070445
376385    0.057914
376435    0.043519
376370    0.072327
376434    0.103075
376459    0.143374
376478    0.031351
376473    0.080421
376484    0.036792
376482    0.043524
376480    0.